# LSTM Multi-Feature Stock Price Prediction with Keras

# Overview

This is a daily Apple stock price forecast project using LSTM with four features inputs and one feature output. 


# Background for LSTM
The long short-term memory (LSTM) unit is an improved version of gated recurrent unit (GRU), which tries to resolve the [vanishing gradient problem](http://neuralnetworksanddeeplearning.com/chap5.html) and keep the long term "memory" activated.

See my other [project](https://github.com/ginochen/LSTM/blob/master/LSTM_min_temp.ipynb) for a picture summary on the network architecture.

# Data 
Daily data of [AAPL](https://finance.yahoo.com/quote/AAPL/history?p=AAPL) and [SPY](https://finance.yahoo.com/quote/SPY/history?p=SPY&.tsrc=fin-srch) downloaded from Yahoo Finance.

Then create batches of data with a generator:

In [253]:
class KerasBatchGenerator(object):

#    def __init__(self, data, num_steps, batch_size, vocabulary, skip_step=5):
    def __init__(self, data, num_steps, batch_size, ndim, skip_step=5):
        self.data = data
        self.num_steps = num_steps # forecast time steps
        self.batch_size = batch_size 
        self.ndim = ndim # number of feature dimensions
        # this will track the progress of the batches sequentially through the
        # data set - once the data reaches the end of the data set it will reset
        # back to zero
        self.current_idx = 0
        # skip_step is the number of words which will be skipped before the next
        # batch is skimmed from the data set
        self.skip_step = skip_step

    def generate(self):
        x = np.zeros((self.batch_size, self.num_steps, self.ndim))
        y = np.zeros((self.batch_size, self.num_steps,1))# make sure to set the last dim to one element, otherwise Keras won't `fit` the model!!!!!!!!!!!!!!!
        while True:
            for i in range(self.batch_size):
                if self.current_idx + self.num_steps >= len(self.data):
                    # reset the index back to the start of the data set
                    self.current_idx = 0
                x[i,:,:] = self.data[self.current_idx:self.current_idx + self.num_steps,:]
                y[i,:,0]   = self.data[self.current_idx + 1:self.current_idx + self.num_steps + 1,0]
                self.current_idx += self.skip_step                
            yield x, y

The 

In [254]:
import pandas as pd
import numpy as np
datapath = 'data/'
series_aapl = pd.read_csv('data/AAPL.csv')
series_spy = pd.read_csv('data/SPY.csv')
#series = pd.concat([series_aapl['Close'],series_aapl['Volume'],series_spy['Close'],series_spy['Volume']],axis=1)
series = np.stack((series_aapl['Close'].values,series_aapl['Volume'].values,series_spy['Close'].values,series_spy['Volume'].values),axis=1)
n = series.shape[0] # total time series length
nsteps = 10
batchsize = 20
lagt=1
ndim = series.shape[1] # number of features
# split data into 80% training, 10% validating,10% testing
train,valid,test = series[:round(0.8*n),:], series[round(0.8*n):round(0.9*n),:], series[round(0.9*n):,:]
train_data_generator = KerasBatchGenerator(train, nsteps, batchsize, ndim, skip_step=nsteps)
valid_data_generator = KerasBatchGenerator(valid, nsteps, batchsize, ndim, skip_step=nsteps)

#x_train,x_valid        = series[:round(0.8*n)], series[round(0.8*n):round(0.9*n)]
#y_train,y_valid,y_test = series.iloc[:,0][lagt:round(0.8*n)+lagt], series.iloc[:,0][round(0.8*n)+lagt:round(0.9*n)+lagt], series.iloc[:,0][round(0.9*n)+lagt:]
print(train.shape,valid.shape)

print('AAPL and SPY data snippet:\n', train[:5])


#series_goog = pd.read_csv('data/GOOG.csv')
#series_fund = pd.read_csv('nyse/fundamentals.csv')
#print(series_fund.columns)
#print(series_fund[series_fund['Ticker Symbol']=='AAP'])
#print(series_fund[series_fund['Ticker Symbol']=='AAP'].count)
#series_price = pd.read_csv('nyse/prices-split-adjusted.csv', error_bad_lines=False)
#series_price = pd.read_csv('nyse/prices.csv')
#print(series_price[series_price['symbol']=='AAP'])
#series.rename(columns={'Daily minimum temperatures in Melbourne, Australia, 1981-1990':'mint'},inplace=True) # rename minimum temp to 'mint'
#y = pd.to_numeric(series["mint"],downcast='float')
#y.index = pd.DatetimeIndex(start='1981-01-01',end='1990-12-31',freq='d')
#freq=365 # sampling freq
#train, valid = series_price[:freq*9], series_price[freq*9:]
#train.index, valid.index = y.index[:freq*9], y.index[freq*9:]
#print(series_price.head(35), series_fund[]
#series_fund.iloc[series_fund['Ticker Symbol']=='AAPL',:]
#series_fund.iloc[series_fund['Ticker Symbol']=='AAPL']



(1612, 4) (202, 4)
AAPL and SPY data snippet:
 [[4.55171430e+01 7.04396000e+07 1.22489998e+02 1.56107100e+08]
 [4.51542850e+01 9.58860000e+07 1.21610001e+02 1.86621600e+08]
 [4.54328580e+01 9.60568000e+07 1.22099998e+02 2.21387400e+08]
 [4.52357140e+01 9.03210000e+07 1.21639999e+02 1.58017600e+08]
 [4.40042840e+01 1.98961700e+08 1.20199997e+02 2.39068800e+08]]


In [259]:
import keras
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, LSTM, TimeDistributed,Activation
import numpy as np
hdim = 128
nepochs = 500
model = Sequential()
model.add(LSTM(hdim, batch_input_shape=(batchsize,nsteps,ndim),  return_sequences=True)) #input_shape required only at the first layer
#model.add(LSTM(hdim, return_sequences=True)) 
# units: hidden nodes
# Input: (batch_size, timesteps, input_dim) # input_shape takes only (timesteps, input_dim) without batch_size
# Output: (batch_size, timesteps, units) if return_sequences=True, else, 2D tensor with shape `(batch_size, units)`
# set return_sequences to True to return the 
# full history of hidden state outputs at all times (i.e. the shape of output is (n_samples, n_timestamps, n_outdims)), 
# or the return value contains only the output at the last timestamp (i.e. the shape will be (n_samples, n_outdims)), 
# which is invalid as the input of the next LSTM layer. 
#
# lstm1, state_h, state_c = LSTM(128, return_sequences=True, return_state=True) 
# with return_state set to true, this returns the sequential hidden states, final hidden state and final cell states. 
model.add(Dropout(0.2))
model.add(LSTM(hdim,return_sequences=True)) # no need to specify input_shape anymore
model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(1))) # y(batchsize,nsteps)
#model.add(Dense(1, activation='linear')) # essentially applies 'linear' activation, a(x)=x, which returns the Dense output directly
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
# metrics: shows how the accuracy is improving during training
checkpointer = keras.callbacks.ModelCheckpoint(filepath=datapath + '/model-{epoch:02d}.hdf5', verbose=10)
# include the epoch in its naming of the model, which is good for keeping track of things.
model.summary()
model.fit_generator(train_data_generator.generate(), len(train)//(batchsize*nsteps), nepochs,
                        validation_data=valid_data_generator.generate(),
                        validation_steps=len(valid)//(batchsize*nsteps), callbacks=[checkpointer])

#model.fit(x_train, y_train, batch_size=batchsize, epochs=nepochs)
#score = model.evaluate(x_valid, y_valid, batch_size=16)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_170 (LSTM)              (20, 10, 128)             68096     
_________________________________________________________________
dropout_158 (Dropout)        (20, 10, 128)             0         
_________________________________________________________________
lstm_171 (LSTM)              (20, 10, 128)             131584    
_________________________________________________________________
dropout_159 (Dropout)        (20, 10, 128)             0         
_________________________________________________________________
time_distributed_34 (TimeDis (20, 10, 1)               129       
Total params: 199,809
Trainable params: 199,809
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
8/8 [==============================] - 17s 2s/step - loss: 8347.3060 - acc: 0.0000e+00 - val_loss: 24256.0273 - val_acc: 0.0

Epoch 38/500
8/8 [==============================] - 0s 36ms/step - loss: 2465.3373 - acc: 0.0000e+00 - val_loss: 12244.9219 - val_acc: 0.0000e+00

Epoch 00038: saving model to data//model-38.hdf5
Epoch 39/500
8/8 [==============================] - 0s 39ms/step - loss: 2207.6099 - acc: 0.0000e+00 - val_loss: 12072.3262 - val_acc: 0.0000e+00

Epoch 00039: saving model to data//model-39.hdf5
Epoch 40/500
8/8 [==============================] - 0s 35ms/step - loss: 2543.3890 - acc: 0.0000e+00 - val_loss: 11902.9609 - val_acc: 0.0000e+00

Epoch 00040: saving model to data//model-40.hdf5
Epoch 41/500
8/8 [==============================] - 0s 34ms/step - loss: 1668.2130 - acc: 0.0000e+00 - val_loss: 11743.3945 - val_acc: 0.0000e+00

Epoch 00041: saving model to data//model-41.hdf5
Epoch 42/500
8/8 [==============================] - 0s 37ms/step - loss: 1808.8382 - acc: 0.0000e+00 - val_loss: 11603.5879 - val_acc: 0.0000e+00

Epoch 00042: saving model to data//model-42.hdf5
Epoch 43/500
8/8 [==

Epoch 80/500
8/8 [==============================] - 0s 40ms/step - loss: 897.5016 - acc: 0.0000e+00 - val_loss: 7731.9282 - val_acc: 0.0000e+00

Epoch 00080: saving model to data//model-80.hdf5
Epoch 81/500
8/8 [==============================] - 0s 36ms/step - loss: 920.9500 - acc: 0.0000e+00 - val_loss: 7666.7827 - val_acc: 0.0000e+00

Epoch 00081: saving model to data//model-81.hdf5
Epoch 82/500
8/8 [==============================] - 0s 37ms/step - loss: 1028.5357 - acc: 0.0000e+00 - val_loss: 7604.7368 - val_acc: 0.0000e+00

Epoch 00082: saving model to data//model-82.hdf5
Epoch 83/500
8/8 [==============================] - 0s 34ms/step - loss: 960.0677 - acc: 0.0000e+00 - val_loss: 7541.5127 - val_acc: 0.0000e+00

Epoch 00083: saving model to data//model-83.hdf5
Epoch 84/500
8/8 [==============================] - 0s 41ms/step - loss: 921.8529 - acc: 0.0000e+00 - val_loss: 7476.0889 - val_acc: 0.0000e+00

Epoch 00084: saving model to data//model-84.hdf5
Epoch 85/500
8/8 [===========

Epoch 122/500
8/8 [==============================] - 0s 36ms/step - loss: 940.6400 - acc: 0.0000e+00 - val_loss: 6004.9585 - val_acc: 0.0000e+00

Epoch 00122: saving model to data//model-122.hdf5
Epoch 123/500
8/8 [==============================] - 0s 37ms/step - loss: 754.4086 - acc: 0.0000e+00 - val_loss: 5974.1396 - val_acc: 0.0000e+00

Epoch 00123: saving model to data//model-123.hdf5
Epoch 124/500
8/8 [==============================] - 0s 45ms/step - loss: 854.9508 - acc: 0.0000e+00 - val_loss: 5959.3984 - val_acc: 0.0000e+00

Epoch 00124: saving model to data//model-124.hdf5
Epoch 125/500
8/8 [==============================] - 0s 44ms/step - loss: 716.8932 - acc: 0.0000e+00 - val_loss: 5934.1299 - val_acc: 0.0000e+00

Epoch 00125: saving model to data//model-125.hdf5
Epoch 126/500
8/8 [==============================] - 0s 42ms/step - loss: 706.6282 - acc: 0.0000e+00 - val_loss: 5911.6533 - val_acc: 0.0000e+00

Epoch 00126: saving model to data//model-126.hdf5
Epoch 127/500
8/8 [=

Epoch 164/500
8/8 [==============================] - 0s 41ms/step - loss: 641.1005 - acc: 0.0000e+00 - val_loss: 5484.5249 - val_acc: 0.0000e+00

Epoch 00164: saving model to data//model-164.hdf5
Epoch 165/500
8/8 [==============================] - 0s 42ms/step - loss: 588.9532 - acc: 0.0000e+00 - val_loss: 5481.3062 - val_acc: 0.0000e+00

Epoch 00165: saving model to data//model-165.hdf5
Epoch 166/500
8/8 [==============================] - 0s 36ms/step - loss: 732.3192 - acc: 0.0000e+00 - val_loss: 5474.9053 - val_acc: 0.0000e+00

Epoch 00166: saving model to data//model-166.hdf5
Epoch 167/500
8/8 [==============================] - 0s 33ms/step - loss: 874.3089 - acc: 0.0000e+00 - val_loss: 5471.6357 - val_acc: 0.0000e+00

Epoch 00167: saving model to data//model-167.hdf5
Epoch 168/500
8/8 [==============================] - 0s 43ms/step - loss: 724.6785 - acc: 6.2500e-04 - val_loss: 5471.2295 - val_acc: 0.0000e+00

Epoch 00168: saving model to data//model-168.hdf5
Epoch 169/500
8/8 [=

...


Epoch 00331: saving model to data//model-331.hdf5
Epoch 332/500
8/8 [==============================] - 0s 37ms/step - loss: 704.3117 - acc: 0.0000e+00 - val_loss: 5245.0024 - val_acc: 0.0000e+00

Epoch 00332: saving model to data//model-332.hdf5
Epoch 333/500
8/8 [==============================] - 0s 39ms/step - loss: 670.5875 - acc: 0.0000e+00 - val_loss: 5253.2705 - val_acc: 0.0000e+00

Epoch 00333: saving model to data//model-333.hdf5
Epoch 334/500
8/8 [==============================] - 0s 37ms/step - loss: 761.7230 - acc: 0.0000e+00 - val_loss: 5273.7617 - val_acc: 0.0000e+00

Epoch 00334: saving model to data//model-334.hdf5
Epoch 335/500
8/8 [==============================] - 0s 39ms/step - loss: 656.8744 - acc: 0.0000e+00 - val_loss: 5281.8521 - val_acc: 0.0000e+00

Epoch 00335: saving model to data//model-335.hdf5
Epoch 336/500
8/8 [==============================] - 0s 37ms/step - loss: 726.0328 - acc: 0.0000e+00 - val_loss: 5285.8818 - val_acc: 0.0000e+00

Epoch 00336: saving


Epoch 00373: saving model to data//model-373.hdf5
Epoch 374/500
8/8 [==============================] - 0s 42ms/step - loss: 766.1580 - acc: 0.0000e+00 - val_loss: 5165.0049 - val_acc: 0.0000e+00

Epoch 00374: saving model to data//model-374.hdf5
Epoch 375/500
8/8 [==============================] - 0s 40ms/step - loss: 731.7083 - acc: 0.0000e+00 - val_loss: 5175.2451 - val_acc: 0.0000e+00

Epoch 00375: saving model to data//model-375.hdf5
Epoch 376/500
8/8 [==============================] - 0s 35ms/step - loss: 674.2998 - acc: 0.0000e+00 - val_loss: 5168.7676 - val_acc: 0.0000e+00

Epoch 00376: saving model to data//model-376.hdf5
Epoch 377/500
8/8 [==============================] - 0s 45ms/step - loss: 809.2854 - acc: 0.0000e+00 - val_loss: 5166.9131 - val_acc: 0.0000e+00

Epoch 00377: saving model to data//model-377.hdf5
Epoch 378/500
8/8 [==============================] - 0s 37ms/step - loss: 789.2823 - acc: 0.0000e+00 - val_loss: 5170.4951 - val_acc: 0.0000e+00

Epoch 00378: saving


Epoch 00415: saving model to data//model-415.hdf5
Epoch 416/500
8/8 [==============================] - 0s 27ms/step - loss: 724.1014 - acc: 0.0000e+00 - val_loss: 5223.5103 - val_acc: 0.0000e+00

Epoch 00416: saving model to data//model-416.hdf5
Epoch 417/500
8/8 [==============================] - 0s 29ms/step - loss: 713.0978 - acc: 0.0000e+00 - val_loss: 5225.1929 - val_acc: 0.0000e+00

Epoch 00417: saving model to data//model-417.hdf5
Epoch 418/500
8/8 [==============================] - 0s 30ms/step - loss: 871.7447 - acc: 0.0000e+00 - val_loss: 5224.0581 - val_acc: 0.0000e+00

Epoch 00418: saving model to data//model-418.hdf5
Epoch 419/500
8/8 [==============================] - 0s 35ms/step - loss: 639.0227 - acc: 0.0000e+00 - val_loss: 5214.7354 - val_acc: 0.0000e+00

Epoch 00419: saving model to data//model-419.hdf5
Epoch 420/500
8/8 [==============================] - 0s 38ms/step - loss: 683.6890 - acc: 0.0000e+00 - val_loss: 5208.7808 - val_acc: 0.0000e+00

Epoch 00420: saving


Epoch 00457: saving model to data//model-457.hdf5
Epoch 458/500
8/8 [==============================] - 0s 36ms/step - loss: 550.2031 - acc: 0.0000e+00 - val_loss: 5344.3857 - val_acc: 0.0000e+00

Epoch 00458: saving model to data//model-458.hdf5
Epoch 459/500
8/8 [==============================] - 0s 38ms/step - loss: 597.2873 - acc: 0.0000e+00 - val_loss: 5362.4897 - val_acc: 0.0000e+00

Epoch 00459: saving model to data//model-459.hdf5
Epoch 460/500
8/8 [==============================] - 0s 37ms/step - loss: 719.8998 - acc: 0.0000e+00 - val_loss: 5366.1938 - val_acc: 0.0000e+00

Epoch 00460: saving model to data//model-460.hdf5
Epoch 461/500
8/8 [==============================] - 0s 37ms/step - loss: 697.4338 - acc: 0.0000e+00 - val_loss: 5360.8955 - val_acc: 0.0000e+00

Epoch 00461: saving model to data//model-461.hdf5
Epoch 462/500
8/8 [==============================] - 0s 37ms/step - loss: 862.3043 - acc: 0.0000e+00 - val_loss: 5361.1089 - val_acc: 0.0000e+00

Epoch 00462: saving


Epoch 00499: saving model to data//model-499.hdf5
Epoch 500/500
8/8 [==============================] - 0s 38ms/step - loss: 715.8935 - acc: 0.0000e+00 - val_loss: 5352.8799 - val_acc: 0.0000e+00

Epoch 00500: saving model to data//model-500.hdf5


# Prediction

In [292]:
#from keras.models import load_model
# load model from single file
#model = load_model('data/model-477.hdf5')
# make predictions
#yhat = model.predict(np.reshape(test[0:10,:],(10,4,1)), batch_size=1, steps=10, verbose=2)
x = np.zeros((1,10,4))
x[0,:,:] = test[0:10,:]
yhat = model.predict(x)
print(yhat)

ValueError: could not broadcast input array from shape (20,10,1) into shape (1,10,1)

# Reference
* [LSTM from Keras](https://keras.io/layers/recurrent/#lstm)
* [Keras LSTM tutorial](http://adventuresinmachinelearning.com/keras-lstm-tutorial/)